# Домашнее задание № 3. Исправление опечаток

## 1. Доп. ранжирование по вероятности (3 балла)

Дополните get_closest_hybrid_match в семинаре так, чтобы из кандадатов с одинаковым расстоянием редактирования выбиралось наиболее вероятное.

In [1]:
# Ячейка импортов

import os, re
from string import punctuation
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
from sklearn.metrics import classification_report
from collections import Counter
import textdistance
from difflib import get_close_matches
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
# Данные

corpus = open('wiki_data.txt', encoding='utf8').read()
vocab = Counter(re.findall('[а-яА-ЯёЁ]+', corpus.lower()))

word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}

vec = CountVectorizer(analyzer='char', ngram_range=(1,1))
X = vec.fit_transform(vocab)

bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

In [5]:
def get_closest_match_with_metric(text, lookup,topn=20, metric=textdistance.levenshtein):
    # Counter можно использовать и с не целыми числами
    similarities = Counter()
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    return similarities.most_common(topn)

# Добавлено косинусное расстояние из семинара
def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])
    
    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn] 
    
    return [(id2word[top], similarities[top]) for top in topn]

def get_closest_hybrid_match(text, X, vec, topn=5, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)
    
    # Добавила выбор между словами с одинаковым расстоянием
    
    distances = []
    for item in closest:
        a = list(item)
        a.append(P(item[0]))
        distances.append(a)
        
    total = pd.DataFrame(distances, columns = ['word', 'distance', 'probability'])
    total = total.sort_values(by = ['distance', 'probability'], ascending = False).reset_index(drop=True)
    
    unique_dist = list(total.distance.unique())
    the_most_probable = []
    for i in range(len(total)):
        if total.distance[i] in unique_dist:
            the_most_probable.append(total.word[i])
            the_most_probable.append(total.distance[i])
            unique_dist.remove(total.distance[i])
            
    return the_most_probable

N = sum(vocab.values())
def P(word, N=N):
    return vocab[word] / N

def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1

In [6]:
# Проверка работы на true и bad
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]
    
    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]
    
    assert len(tokens_1) == len(tokens_2)
    
    return list(zip(tokens_1, tokens_2))

mistakes = []
total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

total = 0
correct = 0

cashed = {}
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if predict_mistaken(pair[1], vocab):
            pred = cashed.get(pair[1], get_closest_hybrid_match(pair[1], X, vec)[0])
            cashed[pair[1]] = pred
        else:
            pred = pair[1]
        
            
        if pred == pair[0]:
            correct += 1
        else:
            mistakes.append((pair[0], pair[1], pred))
        total += 1
            
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] != pred:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == pred:
                mistaken_fixed += 1

In [7]:
# Пример работы

print(get_closest_hybrid_match('кршка', X, vec))

# Результаты

print(f"Процент правильных слов: {round((correct/total)*100, 2)} %")
print(f"Процент исправленных ошибок: {round((mistaken_fixed/total_mistaken)*100, 2)} %")
print(f"Процент ошибочно исправленных правильных слов: {round((correct_broken/total_correct)*100, 2)} %")

['крышка', 0.8333333333333334, 'крка', 0.8, 'корешка', 0.7142857142857143]
Процент правильных слов: 84.5 %
Процент исправленных ошибок: 44.33 %
Процент ошибочно исправленных правильных слов: 9.56 %


**Выводы**

Для сравнения: результаты с семинара, без ранжирования по вероятности:

- Процент правильных слов: 84.2 %
- Процент исправленных ошибок: 42.0 %
- Процент ошибочно исправленных правильных слов: 9.56 %

Таким образом, учитывание вероятности не влияет на ошибочное исправление правильных слов, но на несколько процентов повышает процент исправленных ошибок, так что для улучшения исправления ошибок дополнительное ранжирование по вероятности кажется разумным.

## 2.  Symspell (5 баллов)

Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Там к словам применяется только одна операция - удаление символа. Описание алгоритма по шагам:

1) Составляется словарь правильных слов  
2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово   
3) Для выбора исправления для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений, построенного на шаге 2. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления  
4) Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово  


Оцените качество полученного алгоритма теми же тремя метриками.

In [2]:
# ваш код тут

### 1) Составляется словарь правильных слов

In [8]:
vocab = Counter(re.findall('[а-яА-ЯёЁ]+', corpus.lower()))
vocab_top = {word:count for word, count in vocab.items() if count > 5}

### 2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово

In [9]:
def P(word):
    N = sum(vocab_top.values())
    return np.log(vocab[word] / N)

In [10]:
wrong_right = {}
for word in tqdm(vocab_top.keys()):
    splits = [(word[:j], word[j:]) for j in range(len(word) + 1)]
    for L, R in splits:
        if R:
            delete = L + R[1:]
            if (delete in wrong_right.keys() and P(word) > P(wrong_right[delete])) or (delete not in wrong_right.keys()):
                    wrong_right[delete] = word

  0%|          | 0/68258 [00:00<?, ?it/s]

### 3) Для выбора исправления для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений, построенного на шаге 2. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления

### 4) Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово

In [11]:
def symspell(word):
    variants = delete_letters(word)
    if len(variants) == 0:
        return 0
    else:
        return find_the_best(variants)

def delete_letters(word):
    splits = [(word[:j], word[j:]) for j in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    variants = []
    for example in deletes:
        if example in wrong_right.keys():
            variants.append(wrong_right[example])
    return variants

def find_the_best(variants):
    max_var = P(variants[0])
    best_word = variants[0]
    for var in variants:
        if P(var) > max_var:
            max_var = P(var)
            best_word = var
    return best_word

In [12]:
# Пример
print(symspell("сонце"))
print(symspell("шокалад"))

конце
шоколад


### 5) Оцените качество полученного алгоритма теми же тремя метриками.
1) процент правильных слов;

2) процент исправленных ошибок

3) процент ошибочно исправленных правильных слов

In [13]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]
    
    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]
    
    assert len(tokens_1) == len(tokens_2)
    
    return list(zip(tokens_1, tokens_2))

correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}
for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        
        predicted = cashed.get(pair[1], symspell(pair[1]))
        cashed[pair[1]] = predicted
        
        
        if predicted == pair[0]:
            correct += 1
        total += 1
        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1
                
print(f"Процент правильных слов: {round((correct/total)*100, 2)} %")
print(f"Процент исправленных ошибок: {round((mistaken_fixed/total_mistaken)*100, 2)} %")
print(f"Процент ошибочно исправленных правильных слов: {round((correct_broken/total_correct)*100, 2)} %")

  0%|          | 0/915 [00:00<?, ?it/s]

Процент правильных слов: 36.57 %
Процент исправленных ошибок: 17.62 %
Процент ошибочно исправленных правильных слов: 60.63 %


**Выводы**

Ожидаемо, результаты работы алгоритма ниже, чем у алгоритма Норвига. Так, например, процент исправленных ошибок в 3 раза ниже, чем в алгоритме Норвига (17,6 против 51,2), что совпадает с интуитивными предположениями - в алгоритме Symspell из 4-х возможных вариантов изменений используется только один - удаление, так что ошибки, исправляемые перестановкой, заменой или вставкой не исправляются.

## *3. Чтение. (2 балла)

Прочитайте эту главу в книге Speech and Language Processing - https://web.stanford.edu/~jurafsky/slp3/2.pdf .
Ответьте на следующий вопрос:

1. Что такое Byte-Pair Encoding (опишите по-русски, как минимум 10 предложениями)?

*Это задание не связано напрямую с исправлением опечаток, но это важная тема, к которой мы вернемся в конце курса

   **Byte-Pair Encoding**, или **кодирование пар байтов** – это один из алгоритмов, применяющихся для токенизации текстов. В отличие от более простой токенизации разбиением текста по пробелам (или другим символам), при кодировании пар байтов токены не всегда равны словам, что может быть полезно при работе с редкими или несуществующими словами.
    
   Алгоритм состоит из двух основных шагов. На первом шаге составляется словарь. Изначально он состоит из символов длины 1, например, букв и символа конца слова. Алгоритм проходит по учебному корпусу и считает частотность всех стоящих рядом символов. Самая часто встречающаяся пара символов добавляется в словарь как один символ, после чего в учебном корпусе все эти пары символов заменяются одним, объединенным символом. Затем алгоритм продолжает проходить по учебному корпусу, находя все новые сочетания символов. В результате многократного прогона по корпусу составляется окончательный словарь, в котором к изначальным единичным символам добавлены все возможные сочетания символов, от самых коротких (например, приставок или постфиксов) до полноценных слов.
   
   На следующем шаге применяется парсер токенов, который использует словарь, составленный на первом шаге, для работы с тестовыми данными. Он проходит по тестовым данным в том же порядке, разделяя их на символы в том порядке, в каком эти символы добавлялись в словарь. В результате работы такого алгоритма токенизации большая часть слов из тестовых данных должна быть представлена как полноценные символы (одно слово = один символ), и лишь редкие, незнакомые алгоритму, слова как части: например, символ-приставка + символ-корень + символ-окончание.

